In [1]:
import csv
import os
import pandas as pd
import numpy as np
import requests
import datetime
from datetime import datetime
import requests
import zipfile
import io
import urllib
from sqlalchemy import create_engine
#from sqlalchemy.ext.automap import automap_base
import psycopg2
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import PrimaryKeyConstraint
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.schema import Sequence

In [2]:
# uploads the individual contributor datafile directly from the FEC website. 
# Opens Zipped archive and opens the text file.

r = requests.get("https://www.fec.gov/files/bulk-downloads/2020/indiv20.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
file=z.open('itcont.txt')

In [3]:
columns_dict={'CMTE_ID':'str',
             'AMNDT_IND':'str',
             'RPT_TP':'str',
             'TRANSACTION_PGI':'str',
             'IMAGE_NUM':'str',
             'TRANSACTION_TP':'str',
             'ENTITY_TP':'str',
             'NAME':'str',
             'CITY':'str',
             'STATE':'str',
             'ZIP_CODE':'str',
             'EMPLOYER':'str',
             'OCCUPATION':'str',
             'TRANSACTION_DT':'str',
             'TRANSACTION_AMT':'str',
             'OTHER_ID':'str',
             'TRAN_ID':'str',
             'FILE_NUM':'str',
             'MEMO_CD':'str',
             'MEMO_TEXT':'str',
             "SUB_ID":"str"
            }

columns = list(columns_dict.keys())

In [4]:
#Converts opened text file into a Pandas Dataframe

individual_cont_staging = pd.read_csv(file, sep='|', index_col=False, names=columns, dtype=columns_dict,
                              parse_dates=['TRANSACTION_DT'])

individual_cont_staging.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00432906,T,TER,P,201901219143901218,22Y,IND,"LEMUNYON, GLENN",WASHINGTON,DC,...,NaN,NaN,01182019,500,NaN,SB20A.55756,1305860,NaN,NaN,4021320191639407455
1,C00432906,T,TER,P,201901219143901218,22Y,IND,"LEMUNYON, GLENN",WASHINGTON,DC,...,NaN,NaN,01182019,-500,NaN,SB20A.55755,1305860,NaN,NaN,4021320191639407453
2,C00638478,T,TER,P,201901289144040160,15C,CAN,"JANOWICZ, PHILIP",BUENA PARK,CA,...,FRIENDS OF PHIL JANOWICZ,CANDIDATE,01282019,33000,H8CA39133,3703296,1307800,NaN,CONVERTING PRIMARY LOAN TO CONTRIBUTION,4021220191639267649
3,C00640870,T,TER,P,201901259144002482,15C,CAN,"RICHARDSON, CHARDO",LONGWOOD,FL,...,INFORMATION REQUESTED,INFORMATION REQUESTED,01242019,76,H8FL07054,VTQYWHKD8W6,1307204,NaN,CONTRIBUTION FOR DEBT RETIREMENT,4021320191639532337
4,C00638478,T,TER,P,201901289144040158,15,IND,"STEVENS, RICHARD",DIAMOND BAR,CA,...,NOT EMPLOYED,NOT EMPLOYED,01252019,-50,NaN,3703278,1307800,NaN,CHECK LOST,4021220191639267645


In [5]:
# identify the shape of the dataframe.  It's pretty BIG!

individual_cont_staging.shape

(6886602, 21)

In [6]:
#In effort to convert the date text to date-time field, identified one record that would not convert, becuase the data
#is not in correct format.   Removed that record from dataframe.

# individual_cont_staging.set_index('TRANSACTION_DT')
# error_table = individual_cont_staging.loc['SAN DIEGO']
# error_table

weird = individual_cont_staging['TRANSACTION_DT']=='SAN DIEGO'
weird_df=individual_cont_staging[weird].index
individual_cont1=individual_cont_staging.drop(weird_df)


In [7]:
#Convert the "TRANSACTION_DT" to date-time field

individual_cont1['TRANSACTION_DT'] = pd.to_datetime(individual_cont1['TRANSACTION_DT'], format="%m%d%Y")

individual_cont1.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00432906,T,TER,P,201901219143901218,22Y,IND,"LEMUNYON, GLENN",WASHINGTON,DC,...,NaN,NaN,2019-01-18,500,NaN,SB20A.55756,1305860,NaN,NaN,4021320191639407455
1,C00432906,T,TER,P,201901219143901218,22Y,IND,"LEMUNYON, GLENN",WASHINGTON,DC,...,NaN,NaN,2019-01-18,-500,NaN,SB20A.55755,1305860,NaN,NaN,4021320191639407453
2,C00638478,T,TER,P,201901289144040160,15C,CAN,"JANOWICZ, PHILIP",BUENA PARK,CA,...,FRIENDS OF PHIL JANOWICZ,CANDIDATE,2019-01-28,33000,H8CA39133,3703296,1307800,NaN,CONVERTING PRIMARY LOAN TO CONTRIBUTION,4021220191639267649
3,C00640870,T,TER,P,201901259144002482,15C,CAN,"RICHARDSON, CHARDO",LONGWOOD,FL,...,INFORMATION REQUESTED,INFORMATION REQUESTED,2019-01-24,76,H8FL07054,VTQYWHKD8W6,1307204,NaN,CONTRIBUTION FOR DEBT RETIREMENT,4021320191639532337
4,C00638478,T,TER,P,201901289144040158,15,IND,"STEVENS, RICHARD",DIAMOND BAR,CA,...,NOT EMPLOYED,NOT EMPLOYED,2019-01-25,-50,NaN,3703278,1307800,NaN,CHECK LOST,4021220191639267645


In [8]:
# identify the shape of the dataframe.  It's pretty BIG!

individual_cont1.shape

(6886601, 21)

In [9]:
#Change dtypes of columns that should be numeric to floats.

individual_cont1['TRANSACTION_AMT'] = individual_cont1['TRANSACTION_AMT'].astype(float)

individual_cont1['FILE_NUM'] = individual_cont1['FILE_NUM'].astype(float)

individual_cont1['TRANSACTION_AMT'] = individual_cont1['TRANSACTION_AMT'].astype(float)

individual_cont1['SUB_ID'] = individual_cont1['SUB_ID'].astype(float)

#Check dtype of all the columns.

individual_cont1.dtypes

CMTE_ID                    object
AMNDT_IND                  object
RPT_TP                     object
TRANSACTION_PGI            object
IMAGE_NUM                  object
TRANSACTION_TP             object
ENTITY_TP                  object
NAME                       object
CITY                       object
STATE                      object
ZIP_CODE                   object
EMPLOYER                   object
OCCUPATION                 object
TRANSACTION_DT     datetime64[ns]
TRANSACTION_AMT           float64
OTHER_ID                   object
TRAN_ID                    object
FILE_NUM                  float64
MEMO_CD                    object
MEMO_TEXT                  object
SUB_ID                    float64
dtype: object

In [10]:
individual_cont1.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00432906,T,TER,P,201901219143901218,22Y,IND,"LEMUNYON, GLENN",WASHINGTON,DC,...,NaN,NaN,2019-01-18,500.0,NaN,SB20A.55756,1305860.0,NaN,NaN,4.021320e+18
1,C00432906,T,TER,P,201901219143901218,22Y,IND,"LEMUNYON, GLENN",WASHINGTON,DC,...,NaN,NaN,2019-01-18,-500.0,NaN,SB20A.55755,1305860.0,NaN,NaN,4.021320e+18
2,C00638478,T,TER,P,201901289144040160,15C,CAN,"JANOWICZ, PHILIP",BUENA PARK,CA,...,FRIENDS OF PHIL JANOWICZ,CANDIDATE,2019-01-28,33000.0,H8CA39133,3703296,1307800.0,NaN,CONVERTING PRIMARY LOAN TO CONTRIBUTION,4.021220e+18
3,C00640870,T,TER,P,201901259144002482,15C,CAN,"RICHARDSON, CHARDO",LONGWOOD,FL,...,INFORMATION REQUESTED,INFORMATION REQUESTED,2019-01-24,76.0,H8FL07054,VTQYWHKD8W6,1307204.0,NaN,CONTRIBUTION FOR DEBT RETIREMENT,4.021320e+18
4,C00638478,T,TER,P,201901289144040158,15,IND,"STEVENS, RICHARD",DIAMOND BAR,CA,...,NOT EMPLOYED,NOT EMPLOYED,2019-01-25,-50.0,NaN,3703278,1307800.0,NaN,CHECK LOST,4.021220e+18


In [16]:
#Postgres username, password, and database name
ipaddress = 'localhost'
port = '5432'
username = 'postgres'
password = '78f47f33fac44e478456f537e1e20ee4' 
dbname = 'fec_data'
# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'


In [17]:
# Creates Classes which will serve as the anchor points for our Table, loads table to Postgres and uplads the data

# Base = automap_base()
Base = declarative_base()
engine = create_engine(postgres_str)
#Base.prepare(engine, reflect=True)

class Individuals(Base):
    __tablename__ = 'individual_cont'
    aid = Column(Integer, primary_key=True, autoincrement=True)
    CMTE_ID = Column(String(9), nullable=False)
    AMNDT_IND = Column(String(1))
    RPT_TP = Column(String(3))
    TRANSACTION_PGI=Column(String(5))
    IMAGE_NUM = Column(String(18))
    TRANSACTION_TP=Column(String(3))
    ENTITY_TP=Column(String(3))
    NAME = Column(String(200))
    CITY = Column(String(30))
    STATE = Column(String(2))
    ZIP_CODE = Column(String(9))
    EMPLOYER = Column(String)
    OCCUPATION = Column(String)
    TRANSACTION_DT = Column(DateTime)
    TRANSACTION_AMT = Column(Float(14))
    OTHER_ID = Column(String(9))
    TRAN_ID = Column(String(32))
    FILE_NUM = Column(Float(22))
    MEMO_CD = Column(String(1))
    MEMO_TEXT = Column(String(100))
    SUB_ID = Column(Float(19), nullable=False)
#     __table_args__ = (
#         PrimaryKeyConstraint('SUB_ID','TRAN_ID','FILE_NUM','CMTE_ID'),{})
    
Base.metadata.create_all(engine)

#individual_cont1.to_sql('individual_cont', engine, if_exists='append', index=False)

In [27]:
# cleaned csv to file for upload:
file_path = 'data'
table_name = 'indv_cont'
individual_cont1.to_csv(path_or_buf=f'{file_path}/{table_name}.csv', sep=',' )   

In [28]:
# connect and load csv
conn = psycopg2.connect(f"dbname={dbname} user={username} password={password}")

table = 'individual_cont'
sql = f"COPY {table} FROM STDIN WITH CSV HEADER DELIMITER AS ','"
file = open(f'{file_path}/{table_name}.csv', "r")

with conn.cursor() as cur:
    # truncate is a potentialy dangerous function, but we might
    # run this cell several times. This wipes the tables clean so we 
    # don't append repeated data
    cur.execute(f'truncate {table} cascade;')
    cur.copy_expert(sql=f'{sql}', file=file)
    conn.commit()
    cur.close()